# Wild Blueberry Yield Prediction

**Context**: The dataset used for predictive modeling was generated by the Wild Blueberry Pollination Simulation Model, which is an open-source, spatially-explicit computer simulation program that enables exploration of how various factors, including plant spatial arrangement, outcrossing and self-pollination, bee species compositions and weather conditions, in isolation and combination, affect pollination efficiency and yield of the wild blueberry agroecosystem.

**Goal**: Through the provided features predict the wild blueberry yield amount.

**Feature Description**:
- Clonesize: m2 The average blueberry clone size in the field
- Honeybee: bees/m2/min Honeybee density in the field
- Bumbles: bees/m2/min Bumblebee density in the field
- Andrena: bees/m2/min Andrena bee density in the field
- Osmia: bees/m2/min Osmia bee density in the field
- MaxOfUpperTRange: ℃ The highest record of the upper band daily air temperature during the bloom season
- MinOfUpperTRange: ℃ The lowest record of the upper band daily air temperature
- AverageOfUpperTRange: ℃ The average of the upper band daily air temperature
- MaxOfLowerTRange: ℃ The highest record of the lower band daily air temperature
- MinOfLowerTRange: ℃ The lowest record of the lower band daily air temperature
- AverageOfLowerTRange: ℃ The average of the lower band daily air temperature
- RainingDays: Day The total number of days during the bloom season, each of which has precipitation larger than zero
- AverageRainingDays: Day The average of raining days of the entire bloom season

**Resources**:
- [Kaggle Challenge](https://www.kaggle.com/competitions/playground-series-s3e14/overview)
- [Dataset](https://www.kaggle.com/datasets/shashwatwork/wild-blueberry-yield-prediction-dataset)

In [ ]:
# Import Standard Libraries
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from colorama import Style, Fore

In [ ]:
# Define Seaborn theme parameters
theme_parameters =  {
    'axes.spines.right': False,
    'axes.spines.top': False,
    'grid.alpha':0.3,
    'figure.figsize': (16, 6),
    'font.family': 'Andale Mono',
    'axes.titlesize': 24,
    'figure.facecolor': '#E5E8E8',
    'axes.facecolor': '#E5E8E8'
}

# Set the theme
sns.set_theme(style='whitegrid',
              palette=sns.color_palette('deep'), 
              rc=theme_parameters)

In [ ]:
# Define Colors
black = Style.BRIGHT + Fore.BLACK
magenta = Style.BRIGHT + Fore.MAGENTA
red = Style.BRIGHT + Fore.RED
blue = Style.BRIGHT + Fore.BLUE
reset_colors = Style.RESET_ALL

# Read Data

In [ ]:
# Switch flag for Kaggle Cloud
kaggle = False

In [ ]:
# Read training data
if kaggle:
    
    # Read data from Kaggle FS
    train_data = pd.read_csv('/kaggle/input/playground-series-s3e14/train.csv')
    test_data = pd.read_csv('/kaggle/input/playground-series-s3e14/test.csv')
    
else:
    
    # Define local data file paths
    train_data_file_path = Path(os.path.abspath('')).parents[1] / 'data' / 'S3E14' / 'wild_blueberry_yield_train.csv'
    test_data_file_path = Path(os.path.abspath('')).parents[1] / 'data' / 'S3E14' / 'wild_blueberry_yield_test.csv'
   
    train_data = pd.read_csv(train_data_file_path)
    test_data = pd.read_csv(test_data_file_path)

In [ ]:
train_data.head()

In [ ]:
train_data.info()

# Exploratory Data Analysis (EDA)

## Shapes Information

In [ ]:
# Print shapes information
print(f'{blue}Data Shapes:'
      f'{blue}\n- Train Data  -> {red}{train_data.shape}'
      f'{blue}\n- Test Data   -> {red}{test_data.shape}\n')

## Null Values Information

In [ ]:
# Print null values information
print(f'{blue}Data Null Values:'
      f'{blue}\n- Train Data  -> {red}{train_data.isnull().any().sum()}'
      f'{blue}\n- Test Data   -> {red}{test_data.isnull().any().sum()}\n')

## Feature Distribution

In [ ]:
# Plot the KDE of each feature
figure, ax = plt.subplots(4, 4, figsize=(16, 12))
ax = ax.flatten()

# Fetch the data to plot (exclude the 'id' and 'label' columns)
for index, column_name in enumerate(train_data.columns[1:-1]):
    
    # Plot data
    sns.kdeplot(data=train_data[column_name],
                label='Train',
                ax=ax[index])
    
    sns.kdeplot(data=test_data[column_name],
                label='Test',
                ax=ax[index])
    
    ax[index].set_title(column_name, fontsize=14)
    
    ax[index].tick_params(labelrotation=45)
    
    # Retrieve legend information
    handles = ax[index].get_legend_handles_labels()[0]
    labels = ax[index].get_legend_handles_labels()[1]
    ax[index].legend().remove()

# Set the legend
figure.legend(handles, 
              labels, 
              loc='upper center', 
              bbox_to_anchor=(0.5, 1.03), 
              fontsize=12,
              ncol=3)

plt.tight_layout()

### Honeybee Distribution

In [ ]:
# Define figure and axes
figure, ax = plt.subplots(2, 1, figsize=(16, 9))
ax = ax.flatten()

# Plot the Boxplot of 'honeybee'
sns.boxplot(data=train_data,
            x='honeybee',
            ax=ax[0])

sns.boxplot(data=test_data,
            x='honeybee',
            ax=ax[1])

ax[0].set_title('Train Honeybee Distribution')
ax[1].set_title('Test Honeybee Distribution')

plt.tight_layout()

plt.show()

In [ ]:
# Define figure and axes
figure, ax = plt.subplots(2, 1, figsize=(16, 9))
ax = ax.flatten()

# Plot the Boxplot of 'honeybee'
sns.kdeplot(data=train_data[train_data['honeybee'] < 2.5]['honeybee'],
            label='Train',
            ax=ax[0])

sns.kdeplot(data=test_data[test_data['honeybee'] < 2.5]['honeybee'],
            label='Test',
            color='orange',
            ax=ax[1])

figure.suptitle('Honeybee KDE', fontsize=24)

figure.legend()

plt.tight_layout()

plt.show()

- Drop outlisers in `honeybee`
- Exclude `honeybee`, `bumbles`, `andrena` and `osmia` from Standard Scaler